<a href="https://colab.research.google.com/github/theAyushAT/multi_class_object_counting/blob/main/vehicle_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi class object counting

Ref repo : 

https://github.com/cfotache/pytorch_objectdetecttrack

https://github.com/abewley/sort


Input video link https://drive.google.com/file/d/1pLv-quRmgGPQu3i6fY11D_bxXkZtz1hD/view?usp=sharing


Output Video Link Output video - https://drive.google.com/file/d/1areV8LB93m_VFaqPIVDyN1jlj3ubKY6Q/view?usp=sharing



In [ ]:
# repo https://github.com/cfotache/pytorch_objectdetecttrack
!curl --header 'Host: codeload.github.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://github.com/cfotache/pytorch_objectdetecttrack' --cookie '_octo=GH1.1.171669365.1609144657; logged_in=yes; dotcom_user=theAyushAT; color_mode=%7B%22color_mode%22%3A%22light%22%2C%22light_theme%22%3A%7B%22name%22%3A%22light%22%2C%22color_mode%22%3A%22light%22%7D%2C%22dark_theme%22%3A%7B%22name%22%3A%22dark%22%2C%22color_mode%22%3A%22dark%22%7D%7D; tz=Asia%2FKolkata' --header 'Upgrade-Insecure-Requests: 1' 'https://codeload.github.com/cfotache/pytorch_objectdetecttrack/zip/refs/heads/master' --output 'pytorch_objectdetecttrack-master.zip'

In [ ]:
!unzip /content/pytorch_objectdetecttrack-master.zip

In [ ]:
%cd /content/pytorch_objectdetecttrack-master

/content/pytorch_objectdetecttrack-master


In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2021-06-19 11:51:57--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  71.7MB/s    in 3.5s    

2021-06-19 11:52:01 (67.6 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
!pip install filterpy

video link https://drive.google.com/file/d/1pLv-quRmgGPQu3i6fY11D_bxXkZtz1hD/view?usp=sharing


In [ ]:
# video
!curl --header 'Host: doc-10-b8-docs.googleusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --cookie 'AUTH_n1o21ebub997en8ucnes7jnsdk8r5ipt_nonce=7hjn88hlr00u6' --header 'Upgrade-Insecure-Requests: 1' 'https://doc-10-b8-docs.googleusercontent.com/docs/securesc/dnofsi9g1vmid9d0k9i7iuf4r9kpuvl9/a3isoodjja91tl2go6nv3kakb68373a9/1624104750000/09311022561292723842/09651870378833774979/1QUClOAWRVqNJII1crbCbdStJYzq9Uhfx?e=download&authuser=0&nonce=7hjn88hlr00u6&user=09651870378833774979&hash=8mhppmvrhhdk6asreck8gu4d0pasfa0h' --output 'video.mp4'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [ ]:
from models import *
from utils import *

import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

from PIL import Image

For detction YOLOv3 with Darknet backbone is used

In [ ]:
# load weights and set defaults
config_path='config/yolov3.cfg'
weights_path='yolov3.weights'
class_path='config/coco.names'
img_size=416
conf_thres=0.8
nms_thres=0.4

# load model and put into eval mode
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.cuda()
model.eval()

classes = utils.load_classes(class_path)
Tensor = torch.cuda.FloatTensor

In [ ]:
def detect_image(img):
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
    return detections[0]

Object tracking is performed using SORT.

Ref Repo https://github.com/abewley/sort



In [ ]:
videopath = '/content/pytorch_objectdetecttrack-master/video.mp4'

import cv2
from sort import *
colors=[(255,0,0),(0,255,0),(0,0,255),(255,0,255),(128,0,0),(0,128,0),(0,0,128),(128,0,128),(128,128,0),(0,128,128)]

vid = cv2.VideoCapture(videopath)
mot_tracker = Sort() 



fourcc = cv2.VideoWriter_fourcc(*'XVID')
ret,frame=vid.read()
vw = frame.shape[1]
vh = frame.shape[0]
print ("Video size", vw,vh)
fps = vid.get(cv2.CAP_PROP_FPS) 
outvideo = cv2.VideoWriter(videopath.replace(".mp4", "-det.mp4"),fourcc,int(fps),(vw,vh))

frames = 0
cls_lst = []
id_lst =set ()
map_dict = {}
starttime = time.time()
while(True):
    ret, frame = vid.read()
    if not ret:
        break
    # if frames % 2 ==0:
    #     frames += 1
    #     continue
    frames += 1
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pilimg = Image.fromarray(frame)
    detections = detect_image(pilimg)

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    img = np.array(pilimg)
    pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
    unpad_h = img_size - pad_y
    unpad_w = img_size - pad_x
    if detections is not None:
        tracked_objects = mot_tracker.update(detections.cpu())

        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
            box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
            box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
            y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
            x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])
            color = colors[int(obj_id) % len(colors)]
            # print(classes)
            # print(int(cls_pred))
            cls = classes[int(cls_pred)]
            
            if obj_id not in id_lst:
              cls_lst.append(cls)
              id_lst.add(obj_id)
              count = cls_lst.count(cls)
              map_dict.update({int(obj_id):int(count)})

            
            cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 4)
            cv2.rectangle(frame, (x1, y1-35), (x1+len(cls)*19+80, y1), color, -1)
            cv2.putText(frame, str(cls) + "-" + str(map_dict[int(obj_id)]), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)

    outvideo.write(frame)

    if ch == 27:
        break

totaltime = time.time()-starttime
print(frames, "frames", totaltime/frames, "s/frame")
outvideo.release()

print('number of cars are :',cls_lst.count('car'))
print('number of motorbikes are :',cls_lst.count('motorbike'))
print('number of buses are :',cls_lst.count('bus'))
print('number of trucks are :',cls_lst.count('truck'))
print('number of bicycles are :',cls_lst.count('bicycle'))


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


Video size 1280 720


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


5395 frames 0.06157193921913805 s/frame
number of cars are : 544
number of motorbikes are : 0
number of buses are : 13
number of trucks are : 110
number of bicycles are : 0


Output video - https://drive.google.com/file/d/1areV8LB93m_VFaqPIVDyN1jlj3ubKY6Q/view?usp=sharing
